In [23]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bis_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bis_vals)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool2by2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

x = tf.placeholder(tf.float32,shape=[None,784])

y_true = tf.placeholder(tf.float32,shape=[None,10])

x_image = tf.reshape(x,[-1,28,28,1])

convo_1 = convolutional_layer(x_image,shape = [5,5,1,32])
convo_1_pool = max_pool2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pool,shape = [5,5,32,64])
conv_2_pool = max_pool2by2(convo_2)

convo_2_flat = tf.reshape(conv_2_pool,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,rate=1-hold_prob)

y_pred = normal_full_layer(full_one_dropout,10)

## Cost Function

In [40]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

## Optmizer Function

In [41]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

## Initialising Variables

In [42]:
init = tf.global_variables_initializer()

saver = tf.train.Saver()

## Run session

In [49]:
steps = 5000

with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        if i%100==0:
            print("Currently on step {}".format(i))
            print("Accuracy = ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print((sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))*100,"%")
            print("\n")
    saver.save(sess,'cnn_mnist_final/steps_5000.ckpt')

Currently on step 0
Accuracy = 
8.269999921321869 %


Currently on step 100
Accuracy = 
84.7000002861023 %


Currently on step 200
Accuracy = 
90.25999903678894 %


Currently on step 300
Accuracy = 
92.64000058174133 %


Currently on step 400
Accuracy = 
93.41999888420105 %


Currently on step 500
Accuracy = 
93.77999901771545 %


Currently on step 600
Accuracy = 
95.13999819755554 %


Currently on step 700
Accuracy = 
95.3000009059906 %


Currently on step 800
Accuracy = 
95.7099974155426 %


Currently on step 900
Accuracy = 
96.09000086784363 %


Currently on step 1000
Accuracy = 
96.3699996471405 %


Currently on step 1100
Accuracy = 
96.46000266075134 %


Currently on step 1200
Accuracy = 
96.59000039100647 %


Currently on step 1300
Accuracy = 
96.46999835968018 %


Currently on step 1400
Accuracy = 
96.81000113487244 %


Currently on step 1500
Accuracy = 
96.8999981880188 %


Currently on step 1600
Accuracy = 
97.18000292778015 %


Currently on step 1700
Accuracy = 
97.1300005912